## algorithm design and anlysis-2025 spring  homework 3
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1  

请证明基于比较的排序算法计算复杂度下界为$\Omega(nlogn)$

answer：
![问题1](./fig/hw3q1.png)

## 问题 2 

构建二叉搜索树

> 1. 给定一个数组 [48, 33, 49, 47, 42, 46, 32], 构建一个二叉搜索树存储这些数据，请**绘制**所构建的**二叉搜索树**（标明结点的值）。
> 2. 从第1问的二叉搜索树中删除33，请**绘制**删除33后的**二叉搜索树**（标明结点的值）

- 推荐使用graphviz包绘制树的结构

answer：
![问题1](./fig/hw3q2.png)

## 问题 3 

> 下面是九门课程的开始和结束时间:
>
> [(9:00,12:30), (11:00,14:00), (13:00, 14:30), (9:00,10:30),(13:00, 14:30),(14:00,16:30), (15:00,16:30), (15:00,16:30), (9:00,10:30)]
>
> 请使用贪婪算法为这九门课分配教室，要求在同一天内安排这些课，并且每个教室同一时间只能安排一门课。

**请问最少需要几间教室，罗列出每个教室安排的课程**

idea：该问题要求用最少的教室安排所有课程，确保同一教室的课程时间不重叠。采用贪心算法，按课程开始时间排序后，用最小堆动态维护各教室的最早可用时间：遍历每门课程时，若其开始时间 ≥ 堆顶教室的结束时间，则复用该教室并更新堆；否则分配新教室。最终堆的大小即为最少教室数。算法通过排序（O(n log n)）和堆操作（O(n log k)）实现高效求解，确保全局最优。

In [1]:
import heapq

def min_meeting_rooms(intervals):
    if not intervals:
        return 0
    
    # 按开始时间排序
    intervals.sort(key=lambda x: x[0])
    
    # 最小堆，存储结束时间
    heap = []
    heapq.heappush(heap, intervals[0][1])
    
    for interval in intervals[1:]:
        start, end = interval
        
        # 如果当前课程的开始时间 >= 堆顶的结束时间，可以复用教室
        if start >= heap[0]:
            heapq.heappop(heap)
        
        # 将当前课程的结束时间加入堆
        heapq.heappush(heap, end)
    
    return len(heap)

def assign_classrooms(intervals):
    if not intervals:
        return []
    
    # 按开始时间排序
    sorted_intervals = sorted(intervals, key=lambda x: (x[0], x[1]))
    
    # 最小堆，存储(结束时间, 教室编号)
    heap = []
    classrooms = []
    
    # 分配第一门课到教室1
    classroom_id = 1
    heapq.heappush(heap, (sorted_intervals[0][1], classroom_id))
    classrooms.append((classroom_id, [sorted_intervals[0]]))
    
    for interval in sorted_intervals[1:]:
        start, end = interval
        
        # 如果当前课程的开始时间 >= 堆顶的结束时间，可以复用教室
        if start >= heap[0][0]:
            _, room_id = heapq.heappop(heap)
            # 找到对应的教室，添加课程
            for i, (rid, courses) in enumerate(classrooms):
                if rid == room_id:
                    courses.append(interval)
                    break
            heapq.heappush(heap, (end, room_id))
        else:
            # 需要新教室
            classroom_id += 1
            heapq.heappush(heap, (end, classroom_id))
            classrooms.append((classroom_id, [interval]))
    
    return classrooms

# 课程时间表
intervals = [
    (9.0, 12.5),   # 为了比较方便，使用浮点数表示时间
    (11.0, 14.0),
    (13.0, 14.5),
    (9.0, 10.5),
    (13.0, 14.5),
    (14.0, 16.5),
    (15.0, 16.5),
    (15.0, 16.5),
    (9.0, 10.5)
]

# 计算最少需要多少间教室
min_rooms = min_meeting_rooms(intervals)
print(f"最少需要 {min_rooms} 间教室")

# 分配课程到教室
classroom_assignments = assign_classrooms(intervals)

# 打印每个教室的课程安排
for room in classroom_assignments:
    room_id, courses = room
    print(f"\n教室 {room_id}:")
    for course in courses:
        start, end = course
        # 将浮点数时间转换回时间格式
        start_time = f"{int(start)}:{int((start % 1) * 60):02d}"
        end_time = f"{int(end)}:{int((end % 1) * 60):02d}"
        print(f"  {start_time}-{end_time}")
# your algorithm time complexity is: 
'O(nlogn)'

最少需要 3 间教室

教室 1:
  9:00-10:30
  11:00-14:00
  14:00-16:30

教室 2:
  9:00-10:30
  13:00-14:30
  15:00-16:30

教室 3:
  9:00-12:30
  13:00-14:30
  15:00-16:30


'O(nlogn)'

## 问题 4 

> 爬楼梯问题：假设爬楼梯时你每次只能爬一阶或者爬两阶，问爬上n阶的楼梯，你一共有多少种方法

请设计算法完成该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：爬楼梯问题的解决思路是典型的动态规划应用。通过将问题分解为子问题，并利用子问题的解来构建原问题的解，实现了高效求解。算法的核心在于发现到达第 n 级台阶的方法数与前两级台阶方法数之间的递推关系，即 dp[n]=dp[n−1]+dp[n−2]，这与斐波那契数列的性质一致。

In [6]:
class Solution:
    def climbStairs(self, n: int) -> int:
        if n <= 2:
            return n
        # 初始化dp数组，用来保存每个台阶的方法数
        dp = [0] * (n + 1)
        dp[1] = 1
        dp[2] = 2
        # 从第3个台阶开始，根据递推公式计算方法数
        for i in range(3, n + 1):
            dp[i] = dp[i - 1] + dp[i - 2]
        return dp[n]

# 示例测试
sol = Solution()
print(sol.climbStairs(5))  # 输出: 2
print(sol.climbStairs(10))  # 输出: 3
# your algorithm time complexity is: 
'O(n)'

8
89


## 问题 5 

> 0-1背包问题：现在有4块大理石，每个大理石对应的重量和价值使用一个元组表示, 即（重量，价值），4块大理石的重量和价值为：[(5,10), (4,40), (6,30), (3, 50)], 假设你有一辆最大承重为9的小推车，请问使用这个小推车装大理石的最大价值为多少

请设计一个算法解决该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：

In [2]:
def knapsack(weights, values, capacity):
    n = len(weights)
    dp = [[0] * (capacity + 1) for _ in range(n + 1)]
    
    for i in range(1, n + 1):
        for w in range(1, capacity + 1):
            if weights[i-1] <= w:
                dp[i][w] = max(dp[i-1][w], 
                              dp[i-1][w-weights[i-1]] + values[i-1])
            else:
                dp[i][w] = dp[i-1][w]
    
    return dp[n][capacity]

# 输入数据
weights = [5, 4, 6, 3]
values = [10, 40, 30, 50]
capacity = 9

print(knapsack(weights, values, capacity))  # 输出：90
# your algorithm time complexity is:
'O(n × W),n为物品数量，W为背包容量'

90


'O(n × W),n为物品数量，W为背包容量'

## 问题 6 
给定 `n` 个非负整数表示每个宽度为 `1` 的柱子的高度图，计算按此排列的柱子，下雨之后能接多少雨水。



<div style="text-align:center;width:800px;"> <img alt="" src="./fig/hw3q6.png"> </div>

请分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：为了解决问题，我们使用双指针法，设置 left 和 right 两个指针分别从两端向中间移动，并实时维护左右两侧的最大高度 maxLeft 和 maxRight。每次比较 height[left] 与 height[right]，如果 height[left] < height[right]，则移动 left，否则移动 right。在移动过程中，若当前位置高度小于对应侧的最大高度，说明当前位置可以蓄水，蓄水量分别为 maxLeft - height[left] 或 maxRight - height[right]。选择以当前侧最大高度为基准而非对侧高度，是因为水的高度受限于较矮的一侧，可以通过反证法加以证明。

In [8]:
class Solution:
    def trap(self, height):
        """
        :type height: List[int]
        :rtype: int
        """
        left, right = 0, len(height) - 1
        left_max, right_max = 0, 0
        res = 0
        while left < right:
            if height[left] < height[right]:
                if height[left] >= left_max:
                    left_max = height[left]
                else:
                    res += left_max - height[left]
                left += 1
            else:
                if height[right] >= right_max:
                    right_max = height[right]
                else:
                    res += right_max - height[right]
                right -= 1
                
        return res
# 示例测试
sol = Solution()
print(sol.trap([0,1,0,2,1,0,1,3,2,1,2,1]))  # 输出: 6
print(sol.trap([4,2,0,3,2,5]))  # 输出: 9
# your algorithm time complexity is:
'O(n)'

6
9


## 问题 7 

**股票投资组合优化**： 假设你是一位投资者，想要在不同的股票中分配你的资金，以最大化你的投资回报。每只股票都有不同的预期收益率和风险。你的目标是选择一些股票，使得总投资金额不超过你的预算，并且预期收益最大化。

在这个情况下，你可以将每只股票视为一个“物品”，其重量为投资金额，价值为预期收益率。然后，你可以使用分级背包问题的方法来选择部分股票，以便在预算内获得最大的预期收益。

以下是一个简化的例子：

假设你有以下三只股票可供选择：

1. 股票 A：投资金额 5000 美元，预期收益率 10%
2. 股票 B：投资金额 3000 美元，预期收益率 8%
3. 股票 C：投资金额 2000 美元，预期收益率 12%

请设计算法找到最优投资方案，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：将股票投资组合优化问题建模为0-1背包问题，其中每只股票的投资金额视为“重量”，预期收益率视为“价值”。通过动态规划方法，定义状态dp[i]表示在预算为i时能获得的最大预期收益。初始化dp数组为0，然后遍历每只股票，对于每只股票，从预算上限向下更新dp数组，确保每个状态的最优解。最终，dp[预算]即为最大预期收益。

In [17]:
def max_expected_return(stocks, budget):
    # 初始化 dp 数组，长度为预算 + 1，所有值初始化为 0
    dp = [0] * (budget + 1)
    
    # 遍历每只股票
    for investment, return_rate in stocks:
        # 从预算上限向下更新 dp 数组
        for j in range(budget, investment - 1, -1):
            dp[j] = max(dp[j], dp[j - investment] + return_rate)
    
    # 返回最大预期收益
    return dp[budget]

# 示例测试
stocks = [
    (5000, 10),  # 股票 A：投资金额 5000 美元，预期收益率 10%
    (3000, 8),   # 股票 B：投资金额 3000 美元，预期收益率 8%
    (2000, 12)   # 股票 C：投资金额 2000 美元，预期收益率 12%
]
budget = 7000  # 总预算 7000 美元

max_return = max_expected_return(stocks, budget)
print(f"最大预期收益: {max_return}%")
# your algorithm time complexity is:
'O(n×W)，n 是股票数量，W 是预算'

最大预期收益: 22%


## 问题 8

给你二叉搜索树的根节点 root ，该树中的恰好两个节点的值被错误地交换。请在不改变其结构的情况下，恢复这棵树 。设计算法解决该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

e.g.:
---
1.

<div style="text-align:center;width:200px;"> <img alt="" src="./fig/hw3q9-1.jpg"> </div>
输入：root = [1,3,null,null,2]， 输出为：[3,1,null,null,2]， 

解释：3 不能是 1 的左孩子，因为 3 > 1 。交换 1 和 3 使二叉搜索树有效。

2.

 <div style="text-align:center;width:200px;"> <img alt="" src="./fig/hw3q9-1.jpg"> </div>
输入： root=[3,1,4,null,null,2]， 输出为：[2,1,4,null,null,3]

解释：2 不能在 3 的右子树中，因为 2 < 3 。交换 2 和 3 使二叉搜索树有效。

idea：


In [19]:
#大模型生成
from collections import deque

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def recoverTree(root):
    nodes = []
    
    def inorder(node):
        if not node:
            return
        inorder(node.left)
        nodes.append(node)
        inorder(node.right)
    
    inorder(root)
    
    # 找错误节点
    x = y = None
    for i in range(len(nodes) - 1):
        if nodes[i].val > nodes[i+1].val:
            y = nodes[i+1]
            if x is None:
                x = nodes[i]
            else:
                break
    
    # 交换值
    if x and y:
        x.val, y.val = y.val, x.val

def levelOrder(root):
    if not root:
        return []
    queue = deque([root])
    result = []
    while queue:
        node = queue.popleft()
        if node:
            result.append(node.val)
            queue.append(node.left)
            queue.append(node.right)
        else:
            result.append(None)
    # 去除末尾多余的None
    while result and result[-1] is None:
        result.pop()
    return result

# 示例1
root1 = TreeNode(1)
root1.left = TreeNode(3)
root1.left.right = TreeNode(2)
print("恢复前:", levelOrder(root1))  # 输出: [1, 3, None, None, 2]
recoverTree(root1)
print("恢复后:", levelOrder(root1))  # 输出: [3, 1, None, None, 2]

# 示例2
root2 = TreeNode(3)
root2.left = TreeNode(1)
root2.right = TreeNode(4)
root2.right.left = TreeNode(2)
print("恢复前:", levelOrder(root2))  # 输出: [3, 1, 4, None, None, 2]
recoverTree(root2)
print("恢复后:", levelOrder(root2))  # 输出: [2, 1, 4, None, None, 3]
# your algorithm time complexity is:
'O(n)'

恢复前: [1, 3, None, None, 2]
恢复后: [3, 1, None, None, 2]
恢复前: [3, 1, 4, None, None, 2]
恢复后: [2, 1, 4, None, None, 3]


## 问题 9  

给你一个二叉树的根节点 root ，树中每个节点都存放有一个 0 到 9 之间的数字。每条从根节点到叶节点的路径都代表一个数字：

> 例如，从根节点到叶节点的路径 1 -> 2 -> 3 表示数字 123 。

设计一个算法计算从根节点到叶节点生成的所有数字之和 ，分析算法设计思路，计算时间复杂度，并基于python编程实现

叶节点:是指没有子节点的节点。

e.g.:
---
1.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q10-1.jpg"> </div>

 输入：root = [1,2,3], 输出：25

解释：从根到叶子节点路径 1->2 代表数字 12，1->3 代表数字 13，因此和为 12 + 13 = 25 。

2.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q10-2.jpg"> </div>

 输入：root = [4,9,0,5,1], 输出：1026

解释：从根到叶子节点路径 4->9->5 代表数字 495，4->9->1 代表数字 491，4->0 代表数字 40。因此和为 495 + 491 + 40 = 1026 。

idea：为求解根到叶子的路径数字之和问题，我们采用深度优先搜索（DFS）遍历整棵二叉树。在遍历过程中，维护一个当前路径形成的数字，每到达一个新节点时，将当前数字乘以10并加上节点值，表示路径数字的更新。当遍历到叶子节点时，将当前形成的数字累加到总和中。最终，遍历完整棵树后得到所有路径数字之和。该算法时间复杂度为 O(n)其中n是树中节点数，每个节点仅被访问一次。

In [3]:
# 定义二叉树节点结构
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def sumNumbers(root):
    def dfs(node, current_number):
        if not node:
            return 0
        # 更新当前路径的数字
        current_number = current_number * 10 + node.val
        # 如果是叶子节点，返回当前数字
        if not node.left and not node.right:
            return current_number
        # 递归左右子树并返回总和
        return dfs(node.left, current_number) + dfs(node.right, current_number)

    return dfs(root, 0)

# 示例测试
# 输入 root = [1,2,3]
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)

print(sumNumbers(root))  # 输出：25

# 输入 root = [4,9,0,5,1]
root2 = TreeNode(4)
root2.left = TreeNode(9)
root2.right = TreeNode(0)
root2.left.left = TreeNode(5)
root2.left.right = TreeNode(1)

print(sumNumbers(root2))  # 输出：1026

# your algorithm time complexity is:
'O(n)'

25
1026


'O(n)'

## 问题 10  

给你一个二叉树的根节点 root ， 检查它是否轴对称。

1. 分析算法设计思路，计算时间复杂度，并基于python编程实现
2. \* 设计使用递归和迭代两种方法解决这个问题，计算时间复杂度，并基于python编程实现

e.g.:
---
1.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q11-1.png"> </div>

 输入：root = [1,2,2,3,4,4,3]， 输出：true

解释：二叉树 [1,2,2,3,4,4,3] 对称。

2.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q11-2.png"> </div>

 输入：root = [1,2,2,null,3,null,3]， 输出：false

idea：通过递归判断二叉树是否对称，核心是定义一个辅助函数 isMirror，用来比较左子树和右子树是否互为镜像。比较过程中，若两个节点都为空则视为对称；若只有一个为空或节点值不同，则不对称；否则递归检查左节点的左子树与右节点的右子树、左节点的右子树与右节点的左子树是否对称。整体采用自顶向下的递归方式，时间复杂度为O(n)，其中n是节点数。

In [26]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

class Solution:
    def isSymmetric(self, root: TreeNode) -> bool:
        if not root:
            return True
        return self.isMirror(root.left, root.right)
    
    def isMirror(self, left: TreeNode, right: TreeNode) -> bool:
        if not left and not right:
            return True
        if not left or not right:
            return False
        return (left.val == right.val) and self.isMirror(left.left, right.right) and self.isMirror(left.right, right.left)

# 构建示例树并调用isSymmetric函数
if __name__ == "__main__":
    # 构建第一个示例树
    root1 = TreeNode(1)
    root1.left = TreeNode(2)
    root1.right = TreeNode(2)
    root1.left.left = TreeNode(3)
    root1.left.right = TreeNode(4)
    root1.right.left = TreeNode(4)
    root1.right.right = TreeNode(3)
    
    solution1 = Solution()
    print("第一个示例树是否对称:", solution1.isSymmetric(root1))  # 输出: true

    # 构建第二个示例树
    root2 = TreeNode(1)
    root2.left = TreeNode(2)
    root2.right = TreeNode(2)
    root2.left.left = TreeNode(3)
    root2.right.right = TreeNode(3)
    
    solution2 = Solution()
    print("第二个示例树是否对称:", solution2.isSymmetric(root2))  # 输出: false
# your algorithm time complexity is:
'O(n)'

第一个示例树是否对称: True
运行时间: 0.0
第二个示例树是否对称: False
运行时间: 0.0


## 问题 11  

给你一个由 '1'（陆地）和 '0'（水）组成的的二维网格，请你计算网格中岛屿的数量。岛屿总是被水包围，并且每座岛屿只能由水平方向和/或竖直方向上相邻的陆地连接形成。

此外，你可以假设该网格的四条边均被水包围。

分析算法设计思路，计算时间复杂度，并基于python编程实现

e.g.:
---
1.
输入：grid = [

    ["1","1","1","1","0"],

    ["1","1","0","1","0"], 

    ["1","1","0","0","0"], 

    ["0","0","0","0","0"]  

]

输出：1

2.
输入：grid = [

    ["1","1","0","0","0"],

    ["1","1","0","0","0"],

    ["0","0","1","0","0"],

    ["0","0","0","1","1"]

]

输出：3

idea:利用深度优先搜索（DFS）策略来识别并计数二维网格中的岛屿数量。具体来说，算法遍历网格的每个单元格，一旦发现值为'1'的陆地单元格，就触发一次DFS遍历，将所有相连的陆地单元格标记为已访问，并增加岛屿计数。通过这种方式，算法能够识别出网格中所有的岛屿，并且每次发现一个新的岛屿时，岛屿计数器就增加一。这种方法的时间复杂度为O(MN)，其中M和N分别是网格的行数和列数，因为每个单元格都会被检查一次。

In [27]:
class Solution:
    def numIslands(self, grid):
        if not grid:
            return 0
        
        rows, cols = len(grid), len(grid[0])
        count = 0
        
        def dfs(r, c):
            if r < 0 or r >= rows or c < 0 or c >= cols or grid[r][c] == '0':
                return
            grid[r][c] = '0'  # 标记为已访问
            dfs(r+1, c)
            dfs(r-1, c)
            dfs(r, c+1)
            dfs(r, c-1)
        
        for i in range(rows):
            for j in range(cols):
                if grid[i][j] == '1':
                    count += 1
                    dfs(i, j)
        
        return count

# 示例测试
grid1 = [
    ["1", "1", "1", "1", "0"],
    ["1", "1", "0", "1", "0"],
    ["1", "1", "0", "0", "0"],
    ["0", "0", "0", "0", "0"]
]
grid2 = [
    ["1", "1", "0", "0", "0"],
    ["1", "1", "0", "0", "0"],
    ["0", "0", "1", "0", "0"],
    ["0", "0", "0", "1", "1"]
]

solution = Solution()
print("第一个网格的岛屿数量:", solution.numIslands(grid1))  # 输出: 1
print("第二个网格的岛屿数量:", solution.numIslands(grid2))  # 输出: 3
# your algorithm time complexity is:
'O(M×N)，其中 M 和 N 分别是网格的行数和列数。'

第一个网格的岛屿数量: 1
第二个网格的岛屿数量: 3
